In [36]:
import polars as pl

In [37]:
df = pl.read_json('../dataset/untitled.json', infer_schema_length=None)

In [38]:
df.columns

['ecgKey',
 'meterCategory',
 'rechargeMode',
 'balanceReference',
 'correlationId',
 'meterBalanceBefore',
 'rechargeSource',
 'clientRequestDescription',
 'vendorAccountName',
 'isVendorSale',
 'description',
 'indraExternalTransactionId',
 'status',
 'valueAmount',
 'refundedOn',
 'amountPaid',
 'paymentChannel',
 'asyncRequestId',
 'bankInvoiceExpiry',
 'phoneNumber',
 'paymentAccountName',
 'createdAt',
 'cardScheme',
 'isPaymentCompleted',
 'rechargeCompletedAt',
 'paymentAccountNumber',
 'postedToHubtel',
 'paymentReferenceId',
 'isBatchRecharge',
 'address',
 'rechargeInitiatedAt',
 'batchRechargeId',
 'amount',
 'rechargeToken',
 'tenderedAmount',
 'isChallenged',
 'bmsId',
 'isRechargeFulfilled',
 'paymentProvider',
 'meterDistrict',
 'oltpvFineractClientId',
 'paymentChannelId',
 'accountNumber',
 'id',
 'isOltpvSale',
 'createdByName',
 'paymentCompletedAt',
 'rechargeChannel',
 'paymentProcessorId',
 'postedToHubtelDate',
 'responseCode',
 'retryCount',
 'createdBy',
 'spn

In [39]:
df.shape

(100000, 85)

In [40]:
df['amountPaid'].sample(100).unique

<bound method Series.unique of shape: (100,)
Series: 'amountPaid' [f64]
[
	null
	null
	null
	null
	null
	…
	null
	null
	null
	0.0
	null
]>

In [45]:
# df = df.select(
#     [
#         'accountNumber', 'amount', 'amountAfter', 'amountRefunded', 
#         'createdAt', 'month', 'meterCategory', 'meterDistrict', 
#         'meterRegion', 'paymentChannel', 'paymentProvider', 
#         'rechargeSource', 'status'
#     ]
# )


#meterCategory, status, paymentChannel, meterDistrict, meterRegion, amount, accountNumber
df = df.select(pl.col('meterCategory', 'status', 'paymentChannel', 'meterRegion', 'meterDistrict', 'amount', 'accountNumber', 'createdAt')).filter(pl.col('status') == 'success')

In [46]:
#empty cells in the dataset
df.null_count()

meterCategory,status,paymentChannel,meterRegion,meterDistrict,amount,accountNumber,createdAt
u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,3003,6676,0,0,0


In [65]:
df = df.filter(~pl.all_horizontal(pl.all().is_null()))

In [57]:
df = df.with_columns(pl.col("createdAt")
    .str.slice(0, 26)  # Keep up to microseconds: "YYYY-MM-DDTHH:MM:SS.ssssss"
    .str.strptime(pl.Datetime, "%d-%m-%Y %H:%M:%S%.f"),  # Parse as datetime
)

df

meterCategory,status,paymentChannel,meterRegion,meterDistrict,amount,accountNumber,createdAt
str,str,str,str,str,f64,str,datetime[μs]
"""POSTPAID""","""success""","""mobilemoney""","""CENTRAL""","""Winneba District""",81.0,"""755360056""",2024-01-02 04:29:24.440
"""PREPAID""","""success""","""mobilemoney""","""ACCRA EAST""","""Dodowa District""",100.0,"""754226100""",2024-01-02 04:29:29.710
"""PREPAID""","""success""","""mobilemoney""","""Tema Region""","""Ashaiman District""",50.0,"""711533438""",2024-01-02 04:29:38.367
"""PREPAID""","""success""","""mobilemoney""","""ACCRA EAST""","""Adenta District""",500.0,"""713242383""",2024-01-02 04:29:55.923
"""PREPAID""","""success""","""mobilemoney""","""ACCRA EAST""","""Roman Ridge District""",90.0,"""713745205""",2024-01-02 04:29:56.827
…,…,…,…,…,…,…,…
"""PREPAID""","""success""","""mobilemoney""","""ACCRA WEST""","""Ablekuma District""",20.0,"""745747170""",2024-01-01 21:34:56.657
"""PREPAID""","""success""","""mobilemoney""","""TEMA""","""South Tema District""",20.0,"""712310070""",2024-01-01 21:34:59.427
"""PREPAID""","""success""","""mobilemoney""","""ACCRA EAST""","""Mampong District""",20.0,"""737712591""",2024-01-01 21:35:01.237


In [58]:
df.select(pl.col('amount')).min()

amount
f64
0.02


In [59]:
df = df.with_columns(pl.col('createdAt').dt.hour().alias('hour'))

In [60]:
df.select(pl.col('amount').sum())

amount
f64
1.2770e7


In [64]:
df = df.group_by('hour').agg(pl.col("amount").sum().alias("total_amount"))

df.write_csv('untitled1.csv')